In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

In [10]:
base_url = 'https://www.hanbit.co.kr'
sub_url = '/academy/books/new_book_list.html'
url = base_url + sub_url + '?page=' + str(1)
req =requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

In [11]:
lis = soup.select('li.sub_book_list')
len(lis)

20

In [16]:
li = lis[0]
href = li.find('a')['href']
href

'/academy/books/book_view.html?p_code=B8672666879'

In [19]:
sub_req = requests.get(base_url + href)
sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

In [21]:
title = sub_soup.select_one('.store_product_info_box > h3')
title

<h3>STEM CookBook, 문제 해결력을 키우는 기계설계</h3>

In [25]:
info_lis = sub_soup.select('.info_list > li')
info_dict = {}
for li in info_lis:
  key = li.find('strong').get_text().split(' ')[0]
  val = li.find('span').get_text().strip()
  info_dict[key] = val

In [26]:
info_dict

{'ISBN': '9791156646150',
 '물류코드': '4615',
 '저자': '이상범',
 '출간': '2022-08-05',
 '페이지': '680 쪽'}

In [27]:
author = info_dict['저자']
try:
  translator = info_dict['번역']
except:
  translator = '-'
data = info_dict['출간']
page = int(info_dict['페이지'].split(' ')[0].replace(',',''))
author, translator, data, page

('이상범', '-', '2022-08-05', 680)

In [28]:
price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
price

34000

In [29]:
lines = []
for li in lis:
  href = li.find('a')['href']
  sub_req = requests.get(base_url + href)
  sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
  title = sub_soup.select_one('.store_product_info_box > h3')
  info_lis = sub_soup.select('.info_list > li')
  info_dict = {}
  for li in info_lis:
    key = li.find('strong').get_text().split(' ')[0]
    val = li.find('span').get_text().strip()
    info_dict[key] = val
  author = info_dict['저자']
  try:
    translator = info_dict['번역']
  except:
    translator = '-'
  data = info_dict['출간']
  page = int(info_dict['페이지'].split(' ')[0].replace(',',''))
  price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
  lines.append([title, author, translator, data, page, price])

In [ ]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','가격'])
df

In [32]:
lines = []
for page in range(1,11):
  url = base_url + sub_url + '?page=' + str(1)
  req = requests.get(url)
  soup = BeautifulSoup(req.text, 'html.parser')
  lis = soup.select('li.sub_book_list')

  for li in lis:
    href = li.find('a')['href']
    sub_req = requests.get(base_url + href)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
    title = sub_soup.select_one('.store_product_info_box > h3')
    info_lis = sub_soup.select('.info_list > li')
    info_dict = {}
    for li in info_lis:
      key = li.find('strong').get_text().split(' ')[0]
      val = li.find('span').get_text().strip()
      info_dict[key] = val
    author = info_dict['저자']
    try:
      translator = info_dict['번역']
    except:
      translator = '-'
    data = info_dict['출간']
    page = int(info_dict['페이지'].split(' ')[0].replace(',',''))
    price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
    lines.append([title, author, translator, data, page, price])

In [34]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','페이지','가격'])
df.tail()

,제목,저자,번역,출간일,페이지,가격
195,"[IT CookBook, UI/UX 디자인 이론과 실무 with 어도비 XD]",이영주,-,2022-05-20,512,30000
196,"[STEM@CookBook, 공학 기초수학(2판)]",함남우,-,2022-05-02,484,22000
197,"[STEM@CookBook, Beer의 정역학과 재료역학(3판)]","Ferdinand P. Beer , E. Russell Johnston, Jr. ,...","김경식 , 박재균 , 오창국 , 이정휘",2022-03-22,772,39000
198,[STEIN 푸리에 해석학],"ELIAS M. STEIN , RAMI SHAKARCHI","한빛수학교재연구소 , 이상혁(감수)",2022-03-18,348,38000
199,[미분적분학 : 기초부터 응용까지],수학교재편찬위원회,-,2022-03-04,680,35000
